<a href="https://colab.research.google.com/github/vasanthiginkala5/Hackathon_Submissions/blob/main/Innomatics_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
os.listdir()


['.config', 'sample_data']

In [8]:
from google.colab import files
uploaded = files.upload()


Saving restaurants.sql to restaurants (1).sql
Saving users.json to users (1).json
Saving orders.csv to orders (1).csv


In [9]:
import os
os.listdir()


['.config',
 'orders - Copy.csv',
 'orders.csv',
 'restaurants (1).sql',
 'users (1).json',
 'users.json',
 'restaurants.sql',
 'orders (1).csv',
 'sample_data']

In [ ]:
import os
os.listdir()


['.config',
 'orders - Copy.csv',
 'orders.csv',
 'restaurants (1).sql',
 'users (1).json',
 'users.json',
 'restaurants.sql',
 'orders (1).csv',
 'sample_data']

In [10]:
import pandas as pd
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [11]:
users = pd.read_json("users.json")
users = pd.json_normalize(users)
users.head()


""
0
1
2
3


In [12]:
import sqlite3

conn = sqlite3.connect(":memory:")
with open("restaurants.sql") as f:
    conn.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [15]:
import json

with open("users.json", "r") as f:
    data = json.load(f)

print(type(data))


<class 'list'>


In [16]:
import pandas as pd

users = pd.json_normalize(data)
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [17]:
users.columns


Index(['user_id', 'name', 'city', 'membership'], dtype='object')

In [18]:
df = orders.merge(users, on="user_id", how="left") \
           .merge(restaurants, on="restaurant_id", how="left")

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [19]:
df.to_csv("final_food_delivery_dataset.csv", index=False)
print("✅ final_food_delivery_dataset.csv CREATED SUCCESSFULLY")


✅ final_food_delivery_dataset.csv CREATED SUCCESSFULLY


In [20]:
users.shape
users.columns


Index(['user_id', 'name', 'city', 'membership'], dtype='object')

In [21]:
df.shape


(10000, 12)

In [23]:
df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [ ]:
df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [24]:
df.rename(columns={'membership': 'membership_type'}, inplace=True)


In [25]:
df['membership_type'].value_counts()


,count
membership_type,
Regular,5013
Gold,4987


In [26]:
df[df['membership_type']=="Gold"] \
.groupby('city')['total_amount'] \
.sum().sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [27]:
df.groupby('cuisine')['total_amount'] \
.mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [28]:
user_spend = df.groupby('user_id')['total_amount'].sum()
count = user_spend[user_spend > 1000].count()
count


np.int64(2544)

In [30]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels)

df.groupby('rating_range')['total_amount'] \
   .sum() \
   .sort_values(ascending=False)


/tmp/ipython-input-733902074.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('rating_range')['total_amount'] \


,total_amount
rating_range,
4.6–5.0,2197030.75
4.1–4.5,1960326.26
3.0–3.5,1881754.57
3.6–4.0,1717494.41


In [32]:
df[df['membership_type']=="Gold"] \
.groupby('city')['total_amount'] \
.mean().sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [33]:
df.groupby('cuisine').agg(
    restaurant_count=('restaurant_id','nunique'),
    revenue=('total_amount','sum')
).sort_values('restaurant_count')


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [34]:
gold_pct = round(
    (df[df['membership_type']=="Gold"].shape[0] / df.shape[0]) * 100
)
gold_pct


50

In [37]:
df.groupby('restaurant_name_y').agg(
    avg_order=('total_amount','mean'),
    orders=('order_id','count')
).query('orders < 20') \
  .sort_values('avg_order', ascending=False).head()


,avg_order,orders
restaurant_name_y,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16


In [39]:
df.groupby(['membership_type','cuisine'])['total_amount'] \
.sum().sort_values(ascending=False)


membership_type  cuisine
Regular          Mexican    1072943.30
                 Italian    1018424.75
Gold             Mexican    1012559.79
                 Italian    1005779.05
Regular          Indian      992100.27
Gold             Indian      979312.31
                 Chinese     977713.74
Regular          Chinese     952790.91
Name: total_amount, dtype: float64

In [40]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['quarter'] = df['order_date'].dt.to_period('Q')

df.groupby('quarter')['total_amount'] \
.sum().sort_values(ascending=False)


/tmp/ipython-input-1667303257.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [42]:
df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership_type',
       'restaurant_name_y', 'cuisine', 'rating', 'rating_range', 'quarter'],
      dtype='object')

In [43]:

gold_orders = df[df['membership_type']=="Gold"].shape[0]
print("1️⃣ Total orders by Gold members:", gold_orders)

hyderabad_revenue = round(df[df['city']=="Hyderabad"]['total_amount'].sum())
print("2️⃣ Total revenue in Hyderabad:", hyderabad_revenue)


distinct_users = df['user_id'].nunique()
print("3️⃣ Distinct users with orders:", distinct_users)

avg_gold_order = round(df[df['membership_type']=="Gold"]['total_amount'].mean(), 2)
print("4️⃣ Average order value for Gold members:", avg_gold_order)


orders_high_rating = df[df['rating'] >= 4.5].shape[0]
print("5️⃣ Orders for restaurants with rating ≥ 4.5:", orders_high_rating)


top_city_gold = df[df['membership_type']=="Gold"].groupby('city')['total_amount'].sum().idxmax()
orders_top_city_gold = df[(df['membership_type']=="Gold") & (df['city']==top_city_gold)].shape[0]
print("6️⃣ Orders in top revenue city among Gold members:", orders_top_city_gold)


1️⃣ Total orders by Gold members: 4987
2️⃣ Total revenue in Hyderabad: 1889367
3️⃣ Distinct users with orders: 2883
4️⃣ Average order value for Gold members: 797.15
5️⃣ Orders for restaurants with rating ≥ 4.5: 3374
6️⃣ Orders in top revenue city among Gold members: 1337


In [45]:
df.shape[0]


10000

In [46]:
df['user_id']


,user_id
0,2508
1,2693
2,2084
3,319
4,1064
...,...
9995,2528
9996,2867
9997,522
9998,319


In [47]:
df['user_id'].nunique()


2883

In [48]:
df.shape[0]

10000